In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from llmlab.check_one_round import prepare_one_batch_messages, send_messages, convert_finished_batch, validate_finished_batch
import json
from llmlab.converter.llm_link.NL_manager import NLManager

In [3]:
data = pd.read_excel(r"./exp-easy.xlsx", sheet_name=None)

In [4]:
# prepare the content to be sent 
NLs = list(data["Sheet1"]["experi_filter"])
notes = list(data["Sheet1"]["Notes"])
content = pd.DataFrame(
    {
        "NL": NLs,
        "Notes": notes,
        "chat_history": None,
        "function_info": None,
        "total_errors": None,
        "count": np.zeros(len(NLs)),
        "pass": np.zeros(len(NLs)).astype("bool"),
    }
)
content = content.applymap(lambda x: [] if x is None else x)

In [5]:
for iteration_num in range(3):
    
    # prepare the messages to be sent, update the content file, and save the jsonl to task_file_name
    content, task_file_name = prepare_one_batch_messages(content, model = "gpt-4-1106-preview")
    
    # # actually upload the file to the openAI api
    submitted_task_path, batch_job_id = send_messages(task_file_name)
    # define the path to save the results 
    result_path = f"./{batch_job_id}.jsonl"
    
    # check if the job is finished or not 
    result_path, result = convert_finished_batch(batch_job_id, result_path)
    
    # perform check of the results
    content = validate_finished_batch(content, result_path)
    
    # save the content for the current iteration
    content.to_excel(f"./iter_{iteration_num}.xlsx")
    
    # if everything is right, we should stop the processing 
    if (content["pass"].sum()) == len(content["pass"]):
        break

Current status: in_progress. Checking again in 10 seconds...
Current status: in_progress. Checking again in 10 seconds...
Current status: in_progress. Checking again in 10 seconds...
Current status: in_progress. Checking again in 10 seconds...
Batch process completed.
####################################################################################################
Function: AddSolid 
Property: mass 
before conversion:
mass 53.0 mg
after conversion:
0.053 g
####################################################################################################
Function: AddSolid 
Property: mass 
before conversion:
mass 34.0 mg
after conversion:
0.034 g
####################################################################################################
Function: AddLiquid 
Property: volume 
before conversion:
volume 25.0 mL
after conversion:
25.0 mL
####################################################################################################
Function: AdjustTemperatureTo 
Property: